In [1]:
import h5py as h5
import hdf5storage
import numpy as np
import imageio
import mat73
import os
import cv2
import random
import time
import argparse
from IPython.display import Image

In [2]:
#Read Picture and return it

def readThePicture(picturepath):
    #  open ImageObject
    img = cv2.imread(picturepath, cv2.IMREAD_UNCHANGED)
    #old
    #imageio.plugins.freeimage.download()
    #img=imageio.imread(picturepath) #liest Bild von picturepath
    return(img) #returns picture

In [3]:
def tMO(file,name): #tonemapping the file
    try:
        if (name == 'reinhard'):
            tom = cv2.createTonemapReinhard()
        if (name == 'mantiuk'):
            tom = cv2.createTonemapMantiuk()
        if (name == 'drago'):
            tom = cv2.createTonemapDrago()
        if (name == 'linear'):
            tom = cv2.createTonemap()
    except: 
        print('ToneMapping Error')
    ldr = tom.process(file)
    return ldr

In [4]:
def convert(img, target_type_min, target_type_max): # converts the input array to a target type with the bounderys given
    imin = img.min()
    imax = img.max()
    #print(imin)
    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b)
    return new_img

In [5]:
def savePic(picture,fileName,extention,outPath): #saves the given array as a pictures to the given output path
    outPath = outPath+fileName+'.'+extention
    print(outPath)
    try:
        #old
        #imageio.imwrite(outPath,picture,format=extention)#extention'')#save the Data (path with name and file extention,PictureMatrix,format)
        #new
        cv2.imwrite(outPath,picture)
    except:
        print('Failed while saving picture: '+fileName+' to '+ outPath+' sorry :(')
        print('--------------------')

In [6]:
def cutPatch(begX,begY,endX,endY,picyx):#cuts out a array of a given array
    try: 
        picyx = picyx[begY:endY,begX:endX] #format y,start:End | x,start:End
    except:
        print('FormatMaking Failed')
    return picyx #returns a small part of the pic file

In [7]:
def Randtone_map():
        #a random tonemapping is returned
        rand = random.SystemRandom()
        tmNumber = round((rand.randint(0, 40)/10)) # generates a random tonempaiing nuber 
        try:
            if (tmNumber<=0):
                return 'reinhard' #retruns the name of the tonemapper
            if (tmNumber==1):
                return 'mantiuk'
            if (tmNumber==2):
                return 'drago'
            if (tmNumber>=3):
                return 'linear'
        except:
            print('there was an tmo Error')
#ToDo Output in CSV to later analize

In [8]:
def totalpatchespossible(path,amountOfPictures,extention,px,py,tokonvPic): #calculates the amount of total possible patches of the path you picked 
    arraysize = 0 #zero the output
    amountOfPictures = amountOfPictures - 1
    tokonvPic= (amountOfPictures-(tokonvPic))+1 #generates the stop argument 
    while (amountOfPictures) >= tokonvPic:
        try:
            helping = os.listdir(path)[amountOfPictures].split('.')[1] #reading the File extention
            if ( helping == extention): #only counts files with the same extention
                he = (path+str(os.listdir(path)[amountOfPictures])) #reading path to picture
                print(he) #prints the name of the picture what is currently been read
                readPic = readThePicture(he)
                arraysize = arraysize + (int(readPic.shape[1]/px)*int(readPic.shape[0]/py))# calculate the whole size an cut away the rest even when 0.9
        except:
            print('fail count all patches')
        amountOfPictures = amountOfPictures - 1
    print('All patches will be: '+str(arraysize))
    return arraysize         

In [9]:
def patchesxy(inputpic,px,py): #calculates how often the array can be devided by px in x and py in y
    arraysize = []
    try:
        y = int(inputpic.shape[0]/py)# calculates the number of patches in the Y-axses cuts the picture i
        x = int(inputpic.shape[1]/px)
        arraysize = (x,y)
    except:
        print('fail calc x and y')
    return arraysize
        

In [10]:
def resizePic(inputpic,factor): #reszizing the inputpic picture keeping the information but scaling it down
    y = int((inputpic.shape[0])/factor) #multiply the Factor
    x = int((inputpic.shape[1])/factor) #multiply the Factor
    pic = cv2.resize(inputpic,(x,y)) 
    return pic

In [11]:
def RGBtoYUV(img): #changeing the img picture from RGB- to YUV-Color space
    pictureYUV = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    return pictureYUV

In [12]:
def YUVtoRGB(img):#changeing the img picture from YUV- to RGB-Color space
    pictureRGB = cv2.cvtColor(img, cv2.COLOR_YUV2RGB)
    return pictureRGB

In [13]:
#TO Finish
def inputargs():#todo finish
    parser = argparse.ArgumentParser()
    parser.add_argument('--foo', help='foo help')
    args = parser.parse_args()
    

In [26]:
#---- input section
#TO DO add parser
        
start_time = time.time() #start the timeing of the Prgramm
path = ''

#data extention of the input data
extention = input('What fileextention is supposed to be put into patches(e.g. hdr, png) [default: hdr: ]') or 'hdr'
print(extention)
#Where are the rawData at?
path = input('Where is the Path to the pictures (should be .hdr) [default: ./hdrInput/]') or './hdrInput/'
if not os.path.exists(path):
        os.mkdir(path)
print(path)
inhalte = (os.listdir(path)) #list of all files in path
amountOfPictures = 0

amountOfPictures = sum(1 for f in os.listdir(path) if f.endswith('.'+extention)) #summ all data ending with right extention
print('There are: '+str(amountOfPictures)+' '+extention+' Pictures in the folder')
#User can choose how many pictures should be cut in patches
tokonvPic = int(input('how many Pictures do you want to cut into patches? default 2: ') or '2')
print(str(tokonvPic)+' pictures will be cut into patches')
#scale factor for the low resolution is inputed
factor = int(input('Scale factor for Ldr LR [default:2]: ') or "2")
#asks for the px size of the high resolution pictures
xaxis = int(input('X Patch size HDR HR [default:320px]: ') or "320")
print(xaxis)
yaxis = int(input('Y Patch size HDR HR [default:320px]: ') or "320")
print(yaxis)
#user can choose if the pacht-pictures should be in YU-V or RGB
youWantYUV = input('Do you want to convert to yuv default: yes') or 'yes'
#user can coose in wich folder the .mat file is stored
savein = input('Should patches be saved in .mat file type: (m) oder should ist be saved as pictures (p) or saved as mat and .hdr/.png type: (mp), [default: p] ') or 'p'      
print(savein)
unit_varSdr = (np.float32)
unit_varHdr = (np.float32)
if (savein == 'm' or savein == 'mp'):
    #user can choose the name for the .mat file
    matName = input('Input Mat name default: data ') or 'data'
    matPath = input('Input Mat directory path: ./matOut/') or './matOut/'
    if not os.path.exists(matPath):
            os.mkdir(matPath)
    jsi = input('Is it for the JSI-GAN? default: no') or 'no'
    if (jsi != 'no'):
        unit_varSdr = (np.uint8)
        print('SDR .mat file will be uint8')
        unit_varHdr = (np.uint16)
        print('HDR .mat file will be uint16')
if (savein == 'p' or savein == 'mp'): #if user wants to output pates in picters he can choose where
    outPathsdr = input('spezify the output path of sdr pictures [default: ./sdrOut/] ') or './sdrOut/' #set the picture save path if it is choosen
    if not os.path.exists(outPathsdr):
            os.mkdir(outPathsdr)
    outPathhdr = input('spezify the output path of sdr pictures [default: ./hdrOut/] ') or './hdrOut/' #set the picture save path if it is choosen
    if not os.path.exists(outPathhdr):
        os.mkdir(outPathhdr)
    #TO DO if files should all have the same name or original Filename  

What fileextention is supposed to be put into patches(e.g. hdr, png) [default: hdr: ]
Where is the Path to the pictures (should be .hdr) [default: ./hdrInput/]
hdr
./hdrInput/
There are: 46 hdr Pictures in the folder
how many Pictures do you want to cut into patches? default 2: 
2 pictures will be cut into patches
Scale factor for Ldr LR [default:2]: 
X Patch size HDR HR [default:320px]: 
320
Y Patch size HDR HR [default:320px]: 
320
Do you want to concert to yuv default: yes
Should patches be saved in .mat file type: (m) oder should ist be saved as pictures (p) or saved as mat and .hdr/.png type: (mp), [default: p] m
m
Input Mat name default: data 
Input Mat directory path: ./matOut/
Is it for the JSI-GAN? default: noy


In [27]:
### write pic to .mat and/or .hdr/.png

#Just for general information Data Structure JSI-Gan
###['SDR_data'][79][79][2][39839] dtype=uint8 Strukture .mat Data
###['HDR_data'][79][79][2][39839] dtype=uint16 Strukture .mat Data
    

#---- programm section
allpatches = totalpatchespossible(path,amountOfPictures,extention,xaxis,yaxis,tokonvPic)  #calc all output patches
#print(str(allpatches)+'all Patches')

hdrarray = np.zeros((xaxis,yaxis,3,allpatches))#create empty np array of the size of allpatches
hdrarray = hdrarray.astype(unit_varHdr) #changes the type of np array to uint16
xldr = int(xaxis/factor) #calculates the samler array axes x
yldr = int(yaxis/factor)#calculates the samler array axes y
sdrarray = np.zeros((xldr,yldr,3,allpatches)) # creates the np array for the LR SDR array with new axes
sdrarray = sdrarray.astype(unit_varSdr)#changes the type of np array to uint8
#Arrays are defined in [amountOfallPatchesPossible,x,y,RGB]

print('Processing start')
tokonvPic= (amountOfPictures-tokonvPic)# the amount of pictures cut into pachtes is calculated
allpatchesSave = allpatches #all pachtes are saved
amountOfPictures = amountOfPictures - 1 # needs to count down because while counts to more than in amonut of Pictures
while (amountOfPictures >= tokonvPic): #filling Array with pachtes from high to low, beginning with the hightes Number
    currentFile = os.listdir(path)[amountOfPictures] #currentFile holds the name of the current position file 
    try:
        if (currentFile.split('.')[1] == str(extention)): #checks if file is ending as wanted
            he = (path+str(currentFile))#gives the path and name with extention of the to process file
            print('processing the picture: '+he) #prints it to the user
            originalPicture = readThePicture(he) #reads the currentpicture and saves it
            begX = 1 #startingnumber in wich the cutting starts in X-axes
            endX = xaxis # sets current start pixel position at wich the cutting starts in X
            pxy=patchesxy(originalPicture,xaxis,yaxis) # gives back the length of the current picture (numx,numy) e.g. (3,2)
            px= pxy[0] #saves the max x pos in px
            py= pxy[1] #saves the max y pos in py
            while (px >= begX): # cut until you are at the beginning of the picture X position
                begY = 1 #startingnumber in wich the cutting starts in Y-axes
                endY = yaxis # sets current start pixel position at wich the cutting starts in Y
                x = (begX*xaxis)-xaxis #start patch position in x
                while (py>=begY): # cut until you are at the beginning of the picture Y position
                    y = (begY*yaxis)-yaxis #start patch position in y
                    patch = cutPatch(x,y,endX,endY,originalPicture) #make the patch and return it to the patch (floart64) array
                    tmo = Randtone_map() # returns a random name for the tonemapping Operator to use
                    patchLR = resizePic(patch,factor) #resize picture from Patch with the factor
                    patch = patch/((2 ** 16)-1) # normalize between 0,1 a 16 bit HDR Picture
                    patch = convert(patch,0,1)# normalize between 0,1
                    tmoed = tMO(patchLR,tmo) #tonemap the resized patch with the before chosen Tone Mapping Operator
                    patch_lr = (tmoed/((2 ** 8)-1)) # normalize between 0,1 a 8 bit png picture
                    patch_lr = convert(patch_lr,0,1)# normalize between 0,1
                    #YUV needs to be after tonemaping, because the colors needed to be transormed accurate from HDR TO LDR and afterwards
                    # with the toenmapping operator and afterwards transformed to YUV
                    #both Pictures get transformed in 0,1 space to YUV
                    if (youWantYUV != 'no'): #if change to yuv
                        patch_lr = RGBtoYUV(patch_lr) # changes the color space of lr sdr array
                        patch = RGBtoYUV(patch) # changes the color space of hr hdr array
                    patch_lr = patch_lr * ((2 ** 8)-1) # multiply by 255 so a png picture can be saved and it fits to the uint8 format                     
                    if (savein == 'p' or savein == 'mp'): #save as picture if chosen
                        buildFilename = (currentFile+'_'+str(allpatches-1))#builds output name 
                        # TODO Add a Input for the wanted format
                        #print('saving patch number: '+str(allpatches-1))
                        savePic(patch,buildFilename,'hdr',outPathhdr)#change 'hdr' here for different HDR-picture save
                        savePic(patch_lr,buildFilename,'png',outPathsdr)#chnage 'png' here for different LDR-picture save 
                    p = (allpatches-1) #calcualte current patch position
                    if(unit_varHdr == np.uint16):
                        patch = patch * ((2 ** 16)-1) #HDR is changed so it fitts the unit10 format and can be compared
                        patch_lr = patch_lr.astype(unit_varSdr) #changes the data type in according to JSI Gan spec to uint8 for lr sdr
                        patch = patch.astype(unit_varHdr) #changes the data type in according to JSI Gan spec to uint16 for hr hdr
                    try:
                        sdrarray[:,:,:,p] = patch_lr # try Write the tmoed Patch to sdrarray at current patch position
                    except:
                        print('Error at SDR- Array Writing :..(')
                    try:
                        hdrarray[:,:,:,p] = patch # try Write the hr hdr Patch to hdrarray at current patch position
                    except:
                        print('Error at HDR- Array Writing :..(')
                    begY = begY + 1 #Counts up from 1 to all possible cuts of the current picture in Y
                    endY = endY + yaxis #Counts up the position from endY in yaxis steps
                    allpatches = allpatches - 1 #Counts down all patches of all pictures to 0
                    print('Patch === '+str(allpatches)+' ==> Done')
                begX = begX + 1 #Counts up from 1 to all possible cuts of the current picture in X
                endX = endX + xaxis # Counts up the position from endX in xaxis steps
    except:
        print('Error with data maybe not an .hdr file continuing...')  
    amountOfPictures = amountOfPictures - 1 #counts down current picture pos
if (savein == 'mp' or savein == 'm' ): #only makes a Matlap File if wanted
    try:                  
        # Write TO HDR.Mat File
        h5.get_config().default_file_mode = 'a' #write enable
        matfilehdrdataHDR = {} # make a dictionary to store the MAT data in
        print('HDR Matlab file will have the Format')
        print(hdrarray.shape)
        matfilehdrdataHDR[u'HDR_data'] = hdrarray #save hdr array in that dictonary
        print('Writing HDR_'+matName+'.mat File to: '+ matPath)
        hdf5storage.write(matfilehdrdataHDR, '.', matPath+'HDR_'+matName+'.mat', matlab_compatible=True) #output the .mat data file
        print('Saved the HDR_data.mat File')

        #####Writing SDR .mat 
        #Switches the first with the last array Field 
        matfilesdrdatasdr = {} # make a dictionary to store the MAT data in
        print('HDR Matlab file will have the Format')
        print(sdrarray.shape)
        matfilesdrdatasdr[u'SDR_data'] = sdrarray #save sdr array in that dictonary
        print('Writing SDR_'+matName+'.mat File to: '+ matPath)
        hdf5storage.write(matfilesdrdatasdr, '.', matPath+'SDR_'+matName+'.mat', matlab_compatible=True) #output the .mat data file
        print('Saved the SDR_data.mat File')
    except:
        print('error at writing matlab file sorry :(')

print(str((time.time() - start_time)/60)+' Minutes') #outputs the time in minutes
print('------------------------- Done --------------------')

./hdrInput/sculpture_exhibition_16k.hdr
./hdrInput/theater_02_16k.hdr
All patches will be: 2756
Processing start
processing the picture: ./hdrInput/sculpture_exhibition_16k.hdr
Patch === 2755 ==> Done
Patch === 2754 ==> Done
Patch === 2753 ==> Done
Patch === 2752 ==> Done
Patch === 2751 ==> Done
Patch === 2750 ==> Done
Patch === 2749 ==> Done
Patch === 2748 ==> Done
Patch === 2747 ==> Done
Patch === 2746 ==> Done
Patch === 2745 ==> Done
Patch === 2744 ==> Done
Patch === 2743 ==> Done
Patch === 2742 ==> Done
Patch === 2741 ==> Done
Patch === 2740 ==> Done
Patch === 2739 ==> Done
Patch === 2738 ==> Done
Patch === 2737 ==> Done
Patch === 2736 ==> Done
Patch === 2735 ==> Done
Patch === 2734 ==> Done
Patch === 2733 ==> Done
Patch === 2732 ==> Done
Patch === 2731 ==> Done
Patch === 2730 ==> Done
Patch === 2729 ==> Done
Patch === 2728 ==> Done
Patch === 2727 ==> Done
Patch === 2726 ==> Done
Patch === 2725 ==> Done
Patch === 2724 ==> Done
Patch === 2723 ==> Done
Patch === 2722 ==> Done
Patch =

Patch === 2415 ==> Done
Patch === 2414 ==> Done
Patch === 2413 ==> Done
Patch === 2412 ==> Done
Patch === 2411 ==> Done
Patch === 2410 ==> Done
Patch === 2409 ==> Done
Patch === 2408 ==> Done
Patch === 2407 ==> Done
Patch === 2406 ==> Done
Patch === 2405 ==> Done
Patch === 2404 ==> Done
Patch === 2403 ==> Done
Patch === 2402 ==> Done
Patch === 2401 ==> Done
Patch === 2400 ==> Done
Patch === 2399 ==> Done
Patch === 2398 ==> Done
Patch === 2397 ==> Done
Patch === 2396 ==> Done
Patch === 2395 ==> Done
Patch === 2394 ==> Done
Patch === 2393 ==> Done
Patch === 2392 ==> Done
Patch === 2391 ==> Done
Patch === 2390 ==> Done
Patch === 2389 ==> Done
Patch === 2388 ==> Done
Patch === 2387 ==> Done
Patch === 2386 ==> Done
Patch === 2385 ==> Done
Patch === 2384 ==> Done
Patch === 2383 ==> Done
Patch === 2382 ==> Done
Patch === 2381 ==> Done
Patch === 2380 ==> Done
Patch === 2379 ==> Done
Patch === 2378 ==> Done
Patch === 2377 ==> Done
Patch === 2376 ==> Done
Patch === 2375 ==> Done
Patch === 2374 =

Patch === 2066 ==> Done
Patch === 2065 ==> Done
Patch === 2064 ==> Done
Patch === 2063 ==> Done
Patch === 2062 ==> Done
Patch === 2061 ==> Done
Patch === 2060 ==> Done
Patch === 2059 ==> Done
Patch === 2058 ==> Done
Patch === 2057 ==> Done
Patch === 2056 ==> Done
Patch === 2055 ==> Done
Patch === 2054 ==> Done
Patch === 2053 ==> Done
Patch === 2052 ==> Done
Patch === 2051 ==> Done
Patch === 2050 ==> Done
Patch === 2049 ==> Done
Patch === 2048 ==> Done
Patch === 2047 ==> Done
Patch === 2046 ==> Done
Patch === 2045 ==> Done
Patch === 2044 ==> Done
Patch === 2043 ==> Done
Patch === 2042 ==> Done
Patch === 2041 ==> Done
Patch === 2040 ==> Done
Patch === 2039 ==> Done
Patch === 2038 ==> Done
Patch === 2037 ==> Done
Patch === 2036 ==> Done
Patch === 2035 ==> Done
Patch === 2034 ==> Done
Patch === 2033 ==> Done
Patch === 2032 ==> Done
Patch === 2031 ==> Done
Patch === 2030 ==> Done
Patch === 2029 ==> Done
Patch === 2028 ==> Done
Patch === 2027 ==> Done
Patch === 2026 ==> Done
Patch === 2025 =

Patch === 1720 ==> Done
Patch === 1719 ==> Done
Patch === 1718 ==> Done
Patch === 1717 ==> Done
Patch === 1716 ==> Done
Patch === 1715 ==> Done
Patch === 1714 ==> Done
Patch === 1713 ==> Done
Patch === 1712 ==> Done
Patch === 1711 ==> Done
Patch === 1710 ==> Done
Patch === 1709 ==> Done
Patch === 1708 ==> Done
Patch === 1707 ==> Done
Patch === 1706 ==> Done
Patch === 1705 ==> Done
Patch === 1704 ==> Done
Patch === 1703 ==> Done
Patch === 1702 ==> Done
Patch === 1701 ==> Done
Patch === 1700 ==> Done
Patch === 1699 ==> Done
Patch === 1698 ==> Done
Patch === 1697 ==> Done
Patch === 1696 ==> Done
Patch === 1695 ==> Done
Patch === 1694 ==> Done
Patch === 1693 ==> Done
Patch === 1692 ==> Done
Patch === 1691 ==> Done
Patch === 1690 ==> Done
Patch === 1689 ==> Done
Patch === 1688 ==> Done
Patch === 1687 ==> Done
Patch === 1686 ==> Done
Patch === 1685 ==> Done
Patch === 1684 ==> Done
Patch === 1683 ==> Done
Patch === 1682 ==> Done
Patch === 1681 ==> Done
Patch === 1680 ==> Done
Patch === 1679 =

Patch === 1377 ==> Done
Patch === 1376 ==> Done
Patch === 1375 ==> Done
Patch === 1374 ==> Done
Patch === 1373 ==> Done
Patch === 1372 ==> Done
Patch === 1371 ==> Done
Patch === 1370 ==> Done
Patch === 1369 ==> Done
Patch === 1368 ==> Done
Patch === 1367 ==> Done
Patch === 1366 ==> Done
Patch === 1365 ==> Done
Patch === 1364 ==> Done
Patch === 1363 ==> Done
Patch === 1362 ==> Done
Patch === 1361 ==> Done
Patch === 1360 ==> Done
Patch === 1359 ==> Done
Patch === 1358 ==> Done
Patch === 1357 ==> Done
Patch === 1356 ==> Done
Patch === 1355 ==> Done
Patch === 1354 ==> Done
Patch === 1353 ==> Done
Patch === 1352 ==> Done
Patch === 1351 ==> Done
Patch === 1350 ==> Done
Patch === 1349 ==> Done
Patch === 1348 ==> Done
Patch === 1347 ==> Done
Patch === 1346 ==> Done
Patch === 1345 ==> Done
Patch === 1344 ==> Done
Patch === 1343 ==> Done
Patch === 1342 ==> Done
Patch === 1341 ==> Done
Patch === 1340 ==> Done
Patch === 1339 ==> Done
Patch === 1338 ==> Done
Patch === 1337 ==> Done
Patch === 1336 =

Patch === 1027 ==> Done
Patch === 1026 ==> Done
Patch === 1025 ==> Done
Patch === 1024 ==> Done
Patch === 1023 ==> Done
Patch === 1022 ==> Done
Patch === 1021 ==> Done
Patch === 1020 ==> Done
Patch === 1019 ==> Done
Patch === 1018 ==> Done
Patch === 1017 ==> Done
Patch === 1016 ==> Done
Patch === 1015 ==> Done
Patch === 1014 ==> Done
Patch === 1013 ==> Done
Patch === 1012 ==> Done
Patch === 1011 ==> Done
Patch === 1010 ==> Done
Patch === 1009 ==> Done
Patch === 1008 ==> Done
Patch === 1007 ==> Done
Patch === 1006 ==> Done
Patch === 1005 ==> Done
Patch === 1004 ==> Done
Patch === 1003 ==> Done
Patch === 1002 ==> Done
Patch === 1001 ==> Done
Patch === 1000 ==> Done
Patch === 999 ==> Done
Patch === 998 ==> Done
Patch === 997 ==> Done
Patch === 996 ==> Done
Patch === 995 ==> Done
Patch === 994 ==> Done
Patch === 993 ==> Done
Patch === 992 ==> Done
Patch === 991 ==> Done
Patch === 990 ==> Done
Patch === 989 ==> Done
Patch === 988 ==> Done
Patch === 987 ==> Done
Patch === 986 ==> Done
Patch 

Patch === 666 ==> Done
Patch === 665 ==> Done
Patch === 664 ==> Done
Patch === 663 ==> Done
Patch === 662 ==> Done
Patch === 661 ==> Done
Patch === 660 ==> Done
Patch === 659 ==> Done
Patch === 658 ==> Done
Patch === 657 ==> Done
Patch === 656 ==> Done
Patch === 655 ==> Done
Patch === 654 ==> Done
Patch === 653 ==> Done
Patch === 652 ==> Done
Patch === 651 ==> Done
Patch === 650 ==> Done
Patch === 649 ==> Done
Patch === 648 ==> Done
Patch === 647 ==> Done
Patch === 646 ==> Done
Patch === 645 ==> Done
Patch === 644 ==> Done
Patch === 643 ==> Done
Patch === 642 ==> Done
Patch === 641 ==> Done
Patch === 640 ==> Done
Patch === 639 ==> Done
Patch === 638 ==> Done
Patch === 637 ==> Done
Patch === 636 ==> Done
Patch === 635 ==> Done
Patch === 634 ==> Done
Patch === 633 ==> Done
Patch === 632 ==> Done
Patch === 631 ==> Done
Patch === 630 ==> Done
Patch === 629 ==> Done
Patch === 628 ==> Done
Patch === 627 ==> Done
Patch === 626 ==> Done
Patch === 625 ==> Done
Patch === 624 ==> Done
Patch === 6

Patch === 291 ==> Done
Patch === 290 ==> Done
Patch === 289 ==> Done
Patch === 288 ==> Done
Patch === 287 ==> Done
Patch === 286 ==> Done
Patch === 285 ==> Done
Patch === 284 ==> Done
Patch === 283 ==> Done
Patch === 282 ==> Done
Patch === 281 ==> Done
Patch === 280 ==> Done
Patch === 279 ==> Done
Patch === 278 ==> Done
Patch === 277 ==> Done
Patch === 276 ==> Done
Patch === 275 ==> Done
Patch === 274 ==> Done
Patch === 273 ==> Done
Patch === 272 ==> Done
Patch === 271 ==> Done
Patch === 270 ==> Done
Patch === 269 ==> Done
Patch === 268 ==> Done
Patch === 267 ==> Done
Patch === 266 ==> Done
Patch === 265 ==> Done
Patch === 264 ==> Done
Patch === 263 ==> Done
Patch === 262 ==> Done
Patch === 261 ==> Done
Patch === 260 ==> Done
Patch === 259 ==> Done
Patch === 258 ==> Done
Patch === 257 ==> Done
Patch === 256 ==> Done
Patch === 255 ==> Done
Patch === 254 ==> Done
Patch === 253 ==> Done
Patch === 252 ==> Done
Patch === 251 ==> Done
Patch === 250 ==> Done
Patch === 249 ==> Done
Patch === 2

In [ ]:
#python3 main.py --phase train --scale_factor 2 --train_data_path_LR_SDR /SDR_data_.mat --train_data_path_HR_HDR /HDR_data_.mat --epoch 5 --batch_size 4

In [ ]:
#python3 train.py --batch_size 100 -d ./hdrData -s ./pathToSaceChepoints 